<a href="https://colab.research.google.com/github/AUT-Student/NN-HW5/blob/main/NN_HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix

import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import Dense, Input, Concatenate, LSTM, SimpleRNN, RNN
import keras.backend as K
from keras.models import Sequential
from keras.callbacks import EarlyStopping

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Dataset

In [10]:
pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp1.data")

,normal
0,\t-1\t-1\t63\t-3\t-1\t0
1,\t0\t0\t62\t-3\t-1\t0
2,\t-1\t-1\t61\t-3\t0\t0
3,\t-1\t-1\t63\t-2\t-1\t0
4,\t-1\t-1\t63\t-3\t-1\t0
...,...
1402,\t-10\t2\t39\t-21\t-24\t5
1403,\t-11\t2\t38\t-24\t-22\t6
1404,\t-12\t3\t23\t-24\t-24\t5
1405,\t-13\t4\t26\t-29\t-27\t5


In [3]:
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp1.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp2.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp3.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp4.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp5.data

Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp1.data
To: /content/lp1.data
100% 27.3k/27.3k [00:00<00:00, 218kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp2.data
To: /content/lp2.data
100% 14.6k/14.6k [00:00<00:00, 119kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp3.data
To: /content/lp3.data
100% 14.5k/14.5k [00:00<00:00, 129kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp4.data
To: /content/lp4.data
100% 33.6k/33.6k [00:00<00:00, 275kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp5.data
To: /content/lp5.data
100% 49.2k/49.2k [00:00<00:00, 189kB/s]


In [4]:
normal_label_datasets = ["normal","normal","ok","normal","normal"]

dataset_x = []
dataset_y = []
for i in range(5):
  normal_label = normal_label_datasets[i]
  dataset_i = pd.read_csv(f"/content/lp{i+1}.data", header=None)
  
  new_label = None
  new_data = None

  for j, row in dataset_i.iterrows():
    if j%16==0:
      new_data = []
      
      if row[0] == normal_label:
         new_label = 0
      else:
         new_label = 1

    else:
      new_data.append([float(item) for item in row[0].split()])

      if j%16==15:
        dataset_x.append(new_data)
        dataset_y.append(new_label)

In [5]:
dataset_x, dataset_y = shuffle(dataset_x, dataset_y, random_state=0)

In [6]:
dataset_len = len(dataset_x)

In [7]:
train_x = dataset_x[:int(0.7*dataset_len)]
train_y = dataset_y[:int(0.7*dataset_len)]

valid_x = dataset_x[int(0.7*dataset_len):int(0.8*dataset_len)]
valid_y = dataset_y[int(0.7*dataset_len):int(0.8*dataset_len)]

test_x = dataset_x[int(0.8*dataset_len):]
test_y = dataset_y[int(0.8*dataset_len):]

In [8]:
sum(train_y) * 100 / len(train_y)

71.60493827160494

In [9]:
sum(test_y) * 100 / len(test_y)

77.41935483870968

# Elman

In [24]:
class ElmanCell(keras.layers.Layer):
  # Refrences: https://github.com/keras-team/keras/blob/v2.9.0/keras/layers/rnn/base_rnn.py#L36-L862

  def __init__(self, units):
      super().__init__()
      self.units = units
      self.state_size = units

  def build(self, input_shape):
      self.kernel = self.add_weight(shape=(input_shape[-1], self.units),name='kernel')
      self.recurrent_kernel = self.add_weight( shape=(self.units, self.units), name='recurrent_kernel')
      self.built = True

  def call(self, inputs, states):
      prev_output = states[0]
      h = K.dot(inputs, self.kernel)
      output = h + K.dot(prev_output, self.recurrent_kernel)
      return output, [output]

In [25]:
class ElmanNetwork(keras.Model):
  def __init__(self, elman_cell_number):
    super().__init__()
    self.model = minimal_model = Sequential([
                                            Input((15,6,)),
                                            RNN(ElmanCell(elman_cell_number)),
                                            Dense(2, activation="softmax")
                                ])
  def call(self, inputs):
    return self.model.call(inputs)

In [26]:
elman = ElmanNetwork(128)

In [28]:
elman.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [31]:
elman.fit(x=train_x, y=train_y, epochs=100, batch_size=16, validation_data=(valid_x, valid_y))

Epoch 1/100
21/21 [==============================] - 0s 17ms/step - loss: 0.6274 - accuracy: 0.8117 - val_loss: 6.4649 - val_accuracy: 0.6087
Epoch 2/100
21/21 [==============================] - 0s 11ms/step - loss: 0.6190 - accuracy: 0.7994 - val_loss: 6.6207 - val_accuracy: 0.5870
Epoch 3/100
21/21 [==============================] - 0s 11ms/step - loss: 0.6106 - accuracy: 0.8210 - val_loss: 6.3373 - val_accuracy: 0.6304
Epoch 4/100
21/21 [==============================] - 0s 12ms/step - loss: 0.6316 - accuracy: 0.7963 - val_loss: 6.7648 - val_accuracy: 0.5652
Epoch 5/100
21/21 [==============================] - 0s 17ms/step - loss: 0.6395 - accuracy: 0.8056 - val_loss: 6.3136 - val_accuracy: 0.6304
Epoch 6/100
21/21 [==============================] - 0s 15ms/step - loss: 0.6594 - accuracy: 0.7840 - val_loss: 5.2068 - val_accuracy: 0.5870
Epoch 7/100
21/21 [==============================] - 0s 15ms/step - loss: 0.7968 - accuracy: 0.7778 - val_loss: 5.0548 - val_accuracy: 0.6087
Epoch 

In [32]:
elman.evaluate(x=train_x, y=train_y)
elman.evaluate(x=valid_x, y=valid_y)
elman.evaluate(x=test_x, y=test_y)

3/3 [==============================] - 0s 6ms/step - loss: 4.2789 - accuracy: 0.7527


[4.278876304626465, 0.7526881694793701]

# Old

## Elman Network

In [48]:
class Elman(keras.Model):
  def __init__(self):
    super().__init__(self)

    self.model_partial = Sequential([
                             Input(10, name="Input"),
                             Dense(640, name="Hidden1"),
                             Dense(160, name="Hidden2"),
                             Dense(64, name="Hidden3"),
                             Dense(4, name="Hidden4")
                          ])
                            
    self.model_output = Dense(2, activation="softmax", name="Output")

  def call_recercive(self, input, prev_input):
    if len(prev_input)==0:
      concated = K.concatenate([input, K.constant([0., 0., 0., 0.])])
      concated = tf.expand_dims(concated, axis=0)
    else:
      prev_state = self.call_recercive(prev_input[-1], prev_input[:-1])

      input = tf.expand_dims(input, axis=0)
      concated = K.concatenate([input, prev_state])
    
    state = self.model_partial.call(concated)

    return state

  # def call(self, inputs):
  #   inputs = inputs[0]

  #   input = inputs[-1]
  #   prev_input = inputs[:-1]

  #   state = self.call_recercive(input, prev_input)
  #   output = self.model_output(state)

  #   return output


  def call(self, inputs):
    inputs = inputs[0]

    input = inputs[-1]
    concated = K.concatenate([input, K.constant([0., 0., 0., 0.])])
    concated = tf.expand_dims(concated, axis=0)

    state = self.model_partial.call(concated)
    output = self.model_output(state)

    return output


In [49]:
elman = Elman()

In [50]:
elman.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0003), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [51]:
es_callback = EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)

In [52]:
elman.fit(x=train_x, y=train_y, epochs=100, batch_size=1, validation_data=(valid_x, valid_y), callbacks=[es_callback])

Epoch 1/100
324/324 [==============================] - 1s 3ms/step - loss: 10.2907 - accuracy: 0.6481 - val_loss: 10.1209 - val_accuracy: 0.5217
Epoch 2/100
324/324 [==============================] - 1s 2ms/step - loss: 2.4158 - accuracy: 0.7191 - val_loss: 2.5965 - val_accuracy: 0.4565
Epoch 3/100
324/324 [==============================] - 1s 2ms/step - loss: 2.7790 - accuracy: 0.7006 - val_loss: 1.2626 - val_accuracy: 0.5652
Epoch 4/100
324/324 [==============================] - 1s 2ms/step - loss: 3.0551 - accuracy: 0.6698 - val_loss: 6.5484 - val_accuracy: 0.5000
Epoch 5/100
324/324 [==============================] - 1s 2ms/step - loss: 2.7894 - accuracy: 0.6821 - val_loss: 1.6720 - val_accuracy: 0.6087
Epoch 6/100
324/324 [==============================] - 1s 2ms/step - loss: 1.1848 - accuracy: 0.7253 - val_loss: 1.9578 - val_accuracy: 0.5000
Epoch 7/100
324/324 [==============================] - 1s 3ms/step - loss: 0.6743 - accuracy: 0.7438 - val_loss: 1.4733 - val_accuracy: 0.58

## Jordan Network

In [17]:
class Jordan(keras.Model): 
  def __init__(self):
    super().__init__(self)

    self.model = Sequential([
                             Input(8, name="Input"),
                             Dense(64, name="Hidden1"),
                             Dense(64, name="Hidden2"),
                             Dense(32, name="Hidden3"),
                             Dense(2, activation="softmax", name="Output")
    ])

  def call_recercive(self, input, prev_input):
    if len(prev_input)==0:
      concated = K.concatenate([input, K.constant([0., 0.])])
      concated = tf.expand_dims(concated, axis=0)
    else:
      prev_output = self.call_recercive(prev_input[-1], prev_input[:-1])

      input = tf.expand_dims(input, axis=0)
      concated = K.concatenate([input, prev_output])
    
    output = self.model.call(concated)  
    return output

  def call(self, inputs):
    inputs = inputs[0]

    input = inputs[-1]
    prev_input = inputs[:-1]

    return self.call_recercive(input, prev_input)

In [18]:
jordan = Jordan()

In [19]:
jordan.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0003), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [20]:
es_callback = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

In [21]:
log_dir = "./log"
tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
!rm -r ./log

In [22]:
jordan.fit(x=train_x, y=train_y, epochs=50, batch_size=1, validation_data=(valid_x, valid_y), callbacks=[es_callback, tb_callback])

Epoch 1/50
324/324 [==============================] - 4s 5ms/step - loss: 3.4093 - accuracy: 0.6080 - val_loss: 1.6314 - val_accuracy: 0.4348
Epoch 2/50
324/324 [==============================] - 1s 3ms/step - loss: 1.1526 - accuracy: 0.6944 - val_loss: 1.3833 - val_accuracy: 0.5870
Epoch 3/50
324/324 [==============================] - 1s 3ms/step - loss: 3.9900 - accuracy: 0.6914 - val_loss: 1.8111 - val_accuracy: 0.6087
Epoch 4/50
324/324 [==============================] - 1s 3ms/step - loss: 1.0320 - accuracy: 0.7160 - val_loss: 3.5502 - val_accuracy: 0.4565
Epoch 5/50
324/324 [==============================] - 1s 3ms/step - loss: 0.9003 - accuracy: 0.7315 - val_loss: 1.2702 - val_accuracy: 0.4783
Epoch 6/50
324/324 [==============================] - 1s 3ms/step - loss: 1.4149 - accuracy: 0.7284 - val_loss: 1.4060 - val_accuracy: 0.4783
Epoch 7/50
324/324 [==============================] - 1s 3ms/step - loss: 1.6680 - accuracy: 0.6975 - val_loss: 1.1196 - val_accuracy: 0.5652
Epoch 

In [28]:
%tensorboard --logdir ./log

UsageError: Line magic function `%tensorboard` not found.


In [25]:
jordan.evaluate(x=train_x, y=train_y, batch_size=1)
jordan.evaluate(x=valid_x, y=valid_y, batch_size=1)
jordan.evaluate(x=test_x, y=test_y, batch_size=1)

93/93 [==============================] - 0s 2ms/step - loss: 0.4885 - accuracy: 0.7204


[0.4884602427482605, 0.7204301357269287]

In [26]:
test_predict = [np.argmax(item) for item in jordan.predict(test_x, batch_size=1)]

In [27]:
confusion_matrix(test_y, test_predict)

array([[ 4, 17],
       [ 9, 63]])

## LSTM Network

In [36]:
class KerasLSTM(keras.Model):
  def __init__(self):
    super().__init__()

    self.model = Sequential([
        Input(shape=(15,6,)),
        LSTM(32),
        Dense(2, activation="softmax", name="output")
    ])
  
  def call(self, inputs):
    return self.model(inputs)

In [37]:
keras_lstm = KerasLSTM()

In [38]:
keras_lstm.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [39]:
keras_lstm.fit(x=train_x, y=train_y, epochs=50, batch_size=1, validation_data=(valid_x, valid_y), callbacks=[es_callback, tb_callback])

Epoch 1/50
324/324 [==============================] - 5s 7ms/step - loss: 0.3702 - accuracy: 0.8056 - val_loss: 0.2140 - val_accuracy: 0.9130
Epoch 2/50
324/324 [==============================] - 2s 8ms/step - loss: 0.1829 - accuracy: 0.9321 - val_loss: 0.3969 - val_accuracy: 0.8913
Epoch 3/50
324/324 [==============================] - 2s 8ms/step - loss: 0.1221 - accuracy: 0.9506 - val_loss: 0.1063 - val_accuracy: 0.9783
Epoch 4/50
324/324 [==============================] - 2s 7ms/step - loss: 0.0789 - accuracy: 0.9691 - val_loss: 0.1305 - val_accuracy: 0.9348
Epoch 5/50
324/324 [==============================] - 3s 8ms/step - loss: 0.1144 - accuracy: 0.9506 - val_loss: 0.1380 - val_accuracy: 0.9348
Epoch 6/50
324/324 [==============================] - 2s 7ms/step - loss: 0.0940 - accuracy: 0.9599 - val_loss: 0.1656 - val_accuracy: 0.9130
Epoch 7/50
324/324 [==============================] - 2s 7ms/step - loss: 0.1032 - accuracy: 0.9630 - val_loss: 0.2377 - val_accuracy: 0.9130
Epoch 

## RNN Network

In [42]:
class KerasRNN(keras.Model):
  def __init__(self):
    super().__init__()

    self.model = Sequential([
        Input(shape=(15,6,)),
        SimpleRNN(32),
        Dense(2, activation="softmax", name="output")
    ])
  
  def call(self, inputs):
    return self.model(inputs)

In [43]:
keras_rnn = KerasRNN()

In [46]:
keras_rnn.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [47]:
keras_rnn.fit(x=train_x, y=train_y, epochs=50, batch_size=1, validation_data=(valid_x, valid_y), callbacks=[es_callback, tb_callback])

Epoch 1/50
324/324 [==============================] - 2s 4ms/step - loss: 0.3629 - accuracy: 0.8148 - val_loss: 0.3192 - val_accuracy: 0.8478
Epoch 2/50
324/324 [==============================] - 1s 4ms/step - loss: 0.3153 - accuracy: 0.8364 - val_loss: 0.3028 - val_accuracy: 0.8696
Epoch 3/50
324/324 [==============================] - 1s 3ms/step - loss: 0.3052 - accuracy: 0.8673 - val_loss: 0.3040 - val_accuracy: 0.8261
Epoch 4/50
324/324 [==============================] - 1s 3ms/step - loss: 0.2940 - accuracy: 0.8673 - val_loss: 0.3023 - val_accuracy: 0.8478
Epoch 5/50
324/324 [==============================] - 1s 3ms/step - loss: 0.2767 - accuracy: 0.8889 - val_loss: 0.3068 - val_accuracy: 0.8261
Epoch 6/50
324/324 [==============================] - 1s 3ms/step - loss: 0.2689 - accuracy: 0.8611 - val_loss: 0.3193 - val_accuracy: 0.7826
Epoch 7/50
324/324 [==============================] - 1s 3ms/step - loss: 0.2570 - accuracy: 0.8735 - val_loss: 0.2915 - val_accuracy: 0.8696
Epoch 